# WEEK 9

# Movies CSV Processing

**Question:**  
Given a `movies.csv` file with columns: `Title, Year, Genre, Rating`, clean the data by:  
- Removing leading/trailing spaces in all fields  
- Splitting multiple genres (e.g., Action|Sci-Fi) into separate columns (`Genre1, Genre2, …`)  
- Ensuring `Year` and `Rating` are numeric types  

Save the cleaned dataset to `movies_clean.csv`. Generate a summary text file `genre_stats.txt` with the average rating per genre.


In [6]:
import pandas as pd

df = pd.read_csv("movies.csv")
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
genres_split = df["Genre"].str.split("|", expand=True)
genres_split.columns = [f"Genre{i+1}" for i in range(genres_split.shape[1])]
df = pd.concat([df.drop(columns=["Genre"]), genres_split], axis=1)
df["Year"] = pd.to_numeric(df["Year"])
df["Rating"] = pd.to_numeric(df["Rating"])
df.to_csv("movies_clean.csv", index=False)

print("\nmovies_clean.csv content:")
print(df)

genres = pd.melt(df, id_vars=["Title", "Year", "Rating"], value_vars=genres_split.columns, value_name="Genre")
genres = genres.dropna(subset=["Genre"])
genre_stats = genres.groupby("Genre")["Rating"].mean().reset_index()

with open("genre_stats.txt", "w") as f:
    for _, row in genre_stats.iterrows():
        f.write(f"{row['Genre']} : {row['Rating']:.2f}\n")

print("\ngenre_stats.txt content:")
with open("genre_stats.txt", "r") as f:
    print(f.read())



movies_clean.csv content:
        Title  Year  Rating  Genre1   Genre2
0  The Matrix  1999     8.7  Action   Sci-Fi
1   Inception  2010     8.8  Sci-Fi   Action
2  Fight Club  1999     8.8   Drama     None

genre_stats.txt content:
 Action : 8.80
Action : 8.70
Drama : 8.80
Sci-Fi : 8.75



# Students CSV Processing

**Question:**  
Given a `students.csv` file with columns: `Name, Subject, Marks`, perform the following:  
- Convert student names into Title Case (e.g., alice → Alice)  
- Group marks by subject and calculate:  
  - Average marks per subject  
  - Highest scorer in each subject  

Write the results into `summary.csv` in the format:  
`Subject, Average, Topper, TopperMarks`


In [7]:
import pandas as pd

df = pd.read_csv("students.csv")
df["Name"] = df["Name"].str.title()
avg_marks = df.groupby("Subject")["Marks"].mean().reset_index(name="Average")
topper = df.loc[df.groupby("Subject")["Marks"].idxmax()]
summary = avg_marks.merge(topper, on="Subject")
summary = summary.rename(columns={"Name": "Topper", "Marks": "TopperMarks"})
summary.to_csv("summary.csv", index=False)

print("\nsummary.csv content:")
print(summary)



summary.csv content:
   Subject  Average   Topper  TopperMarks
0     Math     91.5  Charlie           95
1  Science     72.0      Bob           72


# Article Text Processing

**Question:**  
Given a `.txt` file containing an article or book chapter, perform the following:  
- Remove punctuation  
- Convert all text to lowercase  
- Tokenize the text into words  
- Count the frequency of each word and store it in a dictionary  

Write the top 20 most frequent words to `output.txt` in the format:  
`word, frequency`


In [8]:
import string
from collections import Counter

with open("article.txt", "r") as f:
    text = f.read()

text = text.lower().translate(str.maketrans("", "", string.punctuation))
words = text.split()
freq = Counter(words)
top20 = freq.most_common(20)

with open("output.txt", "w") as f:
    for word, count in top20:
        f.write(f"{word},{count}\n")

print("\noutput.txt content:")
with open("output.txt", "r") as f:
    print(f.read())



output.txt content:
data,4
science,2
is,2
the,2
future,2
of,1
bright,1
everywhere,1
but,1
not,1
enough,1
insight,1

